# Extracting likes information from facebook personal data

## Import Data

In this section we should import the liked pages and posts by users in outputs/your_data.

In order to do this:
1. download the personal information of each facebook persona you are researching
2. unzip them in the same folder
3. then run: python scripts/fb_import_likes.py -i /path/to/folder/with/LIKES


In [ ]:
import pandas as pd
import altair as alt
import os
alt.renderers.enable('notebook')
%matplotlib inline
def absoluteFilePaths(directory):
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            if str(f)[0] == '.':
                pass
            else:
                yield os.path.abspath(os.path.join(dirpath, f))


folder = "../outputs/fb/your_data/"
files = absoluteFilePaths(folder)

df_list = []
users = []
data = {}
for f in files:
    df = pd.read_csv(f)
    user = df.actor.iloc[0].replace(" ","").lower()
    data.update({user: df})

print(data.keys())

df = pd.concat(data.values())



df = df.set_index(pd.DatetimeIndex(df['timestamp']))
df1 = df.groupby(['actor', 'reaction', 'source', df.index.date]).count()
df1.index.names = ['actor', 'reaction', 'source', 'date']
df1.columns = ['count']
df1 = df1.reset_index()
df1.date = pd.to_datetime(df1.date)

display(df1.head())

## Plot all reactions

In [ ]:
alt.Chart(df).mark_circle().encode(
    x='timestamp:T',
    y='actor:N',
    color=alt.Color('source:N', legend=None),
    size='count()',
    tooltip=['source:N', 'reaction:N', 'count()']
).properties(
    width=600,
    height=400
).interactive()

## Count of reactions per source by user

In [ ]:
input_dropdown = alt.binding_select(options=list(set(df1.actor.unique())))
selection = alt.selection_single(
    fields=['actor'], bind=input_dropdown, name='Select an')
color = alt.condition(selection,
                      alt.Color('source:N', legend=None),
                      alt.value('lightgray'))

alt.Chart(df1).mark_bar().encode(
    x='date:T',
    y='count:Q',
    color='source:N',
tooltip=['date:T', 'count:N', 'source:N', 'reaction:N']
).add_selection(
    selection
).transform_filter(
    selection
)

## Count of reactions per user by source

In [ ]:
input_dropdown = alt.binding_select(options=list(set(df1.source.unique())))
selection = alt.selection_single(
    fields=['source'], bind=input_dropdown, name='Select an')
color = alt.condition(selection,
                      alt.Color('actor:N', legend=None),
                      alt.value('lightgray'))

alt.Chart(df1).mark_bar().encode(
    x='date:T',
    y='count:Q',
    color='actor:N',
    tooltip='count:N'
).add_selection(
    selection
).transform_filter(
    selection
)

In [ ]:
## Check if all of the users are liking the sources needed
